In [ ]:
# Instalar as dependências necessárias
%pip install boto3 requests fpdf2 urllib3 datetime load_dotenv

import boto3
import urllib3
import requests
from fpdf import FPDF
import json
import os
from datetime import datetime, timedelta
from dotenv import load_dotenv
import json
import re


: 

In [ ]:

# RODAR QUANDO QUISER NOVAS TRANSCRIÇÕES

# Configurações
load_dotenv('/package/config/.env')
bucket_name = os.getenv('BUCKET_S3')
transcription_api_key = os.getenv('API_KEY_TRANSCRIPTIONS')
url = os.getenv('URL_TRANSCRIPTIONS')

# Desabilitar o aviso de requisição insegura
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

def fetch_transcripts():
    try:
        # Obter o horário atual
        current_time = datetime.now()
        print('current_time.hour', current_time.hour)
        if current_time.hour == 9:
            # Se for 9h, 15 horas antes pra buscar a partir das 18h do dia anterior
            current_time_adjusted = current_time - timedelta(hours=15)
        else:
            # Caso contrário, 3 horas antes
            current_time_adjusted = current_time - timedelta(hours=3)

        # Adicionar 3 horas ao horário atual
        current_time_plus_3 = current_time_adjusted + timedelta(hours=3)
        # Converter para o formato ISO 8601
        current_time_iso = current_time_plus_3.isoformat()
        current_time_iso = "2024-10-28T12:37:40.695001"
        print(current_time_iso)


        payload = f"{{\"query\":\"query Transcripts($fromDate: DateTime) {{ transcripts(fromDate: $fromDate) {{ title id dateString }} }}\",\"variables\":{{\"fromDate\":\"{current_time_iso}\"}}}}"
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f"Bearer {transcription_api_key}"
        }

        response = requests.post(os.getenv('URL_TRANSCRIPTIONS'), headers=headers, data=payload, verify=False)

        # Verificar se a requisição foi bem-sucedida
        if response.status_code == 200:
            data = response.json()
            print('data',data)
            transcripts = data['data']["transcripts"]

            # Salvar os dados em um arquivo JSON
            with open('transcripts.json', 'w') as f:
                json.dump(transcripts, f, indent=4)
            print("Ids salvos em 'transcripts.json'.")

        else:
            print('Falha na requisição para o Fireflies.')

    except requests.exceptions.RequestException as e:
        print(f"Erro na requisição: {e}")
    except Exception as e:
        print(f"Erro geral: {e}")

# Chame essa função apenas quando quiser atualizar os dados
# fetch_transcripts()

: 

In [ ]:
s3 = boto3.client('s3', region_name='sa-east-1')
url = os.getenv('URL_TRANSCRIPTIONS')
import re

def lambda_handler():
    # Ler os dados salvos no arquivo JSON
    try:
        fetch_transcripts()
        
        with open('transcripts.json', 'r') as f:
            transcripts = json.load(f)
    except FileNotFoundError:
        print("Arquivo 'transcripts.json' não encontrado. Execute a função 'fetch_transcripts()' primeiro.")
        return

    # Processar cada transcrição e gerar o PDF
    for item in transcripts:
        print('item', item)

        formatted_date = format_date(item['dateString'])
        pdf_content = f"{item['title']} - {formatted_date}\n\n"
        
        # Atualizar o pdf_content com as informações da transcrição
        pdf_content = get_sentences(pdf_content, item['id'])
        title_without_specials = re.sub(r'[^A-Za-z0-9]', '', item['title']) or item['id']

        print('title_without_spaces',title_without_specials)
        pdf_filename = f"{formatted_date}-{title_without_specials}.pdf"
        
        # Gerar o PDF com o conteúdo atualizado
        pdf_path = format_pdf(pdf_content, pdf_filename)

        print(f"PDF gerado em handleer depois fformat_pdf: {pdf_path}")
        
        save_pdf_s3(pdf_path, pdf_filename)

def format_pdf(content, filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Helvetica", size=12)

        # Defina a largura da célula para a largura da página menos as margens
        page_width = pdf.epw  # epw é a largura da área de conteúdo da página

        # Adiciona o conteúdo diretamente ao PDF
        # content = re.sub(r'…', '...', content) 
        content = re.sub(r'[^A-Za-z0-9]', content)
        pdf.multi_cell(page_width, 10, content)

        # Salva o PDF no caminho especificado
        pdf_output = f"/tmp/{filename}"
        pdf.output(pdf_output)
        print('PDF salvo em FORMAT PDF:', pdf_output)
        return pdf_output
    except Exception as e:
        print(f"Erro ao gerar PDF: {e}")
        raise
  
def format_date(date_string):
    try:
        # Converter a string para um objeto datetime
        date_object = datetime.strptime(date_string, "%Y-%m-%dT%H:%M:%S.%fZ")
        formatted_date = date_object.strftime("%d-%m-%Y")
        return formatted_date
    except Exception as e:
        print(f"Erro ao formatar data: {e}")
        raise

def get_sentences(pdf_content, id): 
    print(id)
    payload = f"{{\"query\":\"query Transcript($transcriptId: String!) {{ transcript(id: $transcriptId) {{ title id dateString sentences {{ index speaker_name text }} }} }}\",\"variables\":{{\"transcriptId\":\"{id}\"}}}}"
        
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f"Bearer {transcription_api_key}"
    }

    response = requests.request("POST", url, headers=headers, data=payload, verify=False)
    # Verificar se a requisição foi bem-sucedida
    if response.status_code == 200:
        data = json.loads(response.text)
        transcript = data['data']["transcript"]
        print('transcript', transcript)

        previous_speaker = ""
        for sentence in transcript['sentences']:
            if previous_speaker:
                if sentence['speaker_name'] == previous_speaker:
                    pdf_content += pdf_content.rstrip('\n\n') + f" {sentence['text']}\n\n"
                else:
                    pdf_content += f"{sentence['speaker_name']}: {sentence['text']}\n\n"
            previous_speaker = sentence['speaker_name']
        
        return pdf_content
    else:
        print(f"Erro na requisição: {response.status_code}")
        return pdf_content  # Retornar o conteúdo original se a requisição falhar

def save_pdf_s3(pdf_path, pdf_filename):
            # Salvar o PDF no S3
        s3 = boto3.client('s3', region_name='sa-east-1')
        bucket_name = os.getenv('BUCKET_S3')
        try:
            with open(pdf_path, "rb") as pdf_file:
                s3.put_object(Bucket=bucket_name, Key=pdf_filename, Body=pdf_file)
            print(f"PDF salvo no S3: {pdf_filename}")
        except Exception as e:
            print(f"Erro ao salvar o PDF no S3: {e}")

lambda_handler()

current_time.hour 12
2024-10-28T12:37:40.695001
data {'data': {'transcripts': [{'title': 'Bate papo Zup e Felipe | Cultura', 'id': 'SFs4P0MMJJTU5Udp', 'dateString': '2024-10-29T13:58:44.340Z'}, {'title': 'Entrevista técnica Salesforce - Alfredo', 'id': 'bCYsfWlOSPOtpP0c', 'dateString': '2024-10-29T13:58:44.248Z'}, {'title': '[PlayZUP] Bate-papo Inicial! (Eduardo Basílio)', 'id': 'AnF50wu2fGQr1d4c', 'dateString': '2024-10-29T13:29:09.807Z'}, {'title': '[PlayZUP] Bate-papo Inicial! (Luiz Vassoler)', 'id': 'I0XtpzA6oASgpYHe', 'dateString': '2024-10-29T12:28:24.793Z'}, {'title': 'Danielli Andrade / Zup', 'id': 'CwxYk120PxP2B0ya', 'dateString': '2024-10-29T11:58:45.041Z'}, {'title': 'Bate papo Zup e Rosi', 'id': 'ALLz7flgbPAhgca1', 'dateString': '2024-10-29T11:58:44.065Z'}, {'title': 'Alex Simonetti / Zup', 'id': '6GqpU63kcnOLf1Ia', 'dateString': '2024-10-28T18:59:12.226Z'}, {'title': '[PlayZUP] Bate-papo Inicial! (Lorenzo Lessa Menegaz)', 'id': '4XPGUakkurJ0G4gH', 'dateString': '2024-10-28

In [ ]:

# Configurações
load_dotenv('/package/config/.env')

api_key = os.getenv('API_KEY_TRANSCRIPTIONS')

print(f"API Key: {api_key}")


API Key: 467f3c6a-e4a7-4910-9500-34c302dfa15c
